for running an installed Fluo Accumulo, cd to your fluo-uno directory, then ./bin/uno setup accumulo

my monitor links: (might change for others; check your terminal after setting up) <br>Accumulo Monitor: http://localhost:9995/ <br> Hadoop NameNode: http://localhost:8088/ <br> Hadoop ResourceManager: http://localhost:50070/

(from instructions here: https://github.com/apache/fluo-uno)

In [1]:
using Revise
using D4M

┌ Info: Recompiling stale cache file /home/nathan/.julia/compiled/v1.0/D4M/JhFK0.ji for D4M [ca196bdc-a701-11e8-3d50-3b5cc8577617]
└ @ Base loading.jl:1190


Database capabilities loaded!


In [9]:
dbinit()

be sure to config your settings properly! _TODO dump the config file here; note on how to find your config settings -- you need to go into Accumulo_

_TODO also, extra newlines don't work, fix_

_TODO enter config manually and not from a file_

In [14]:
DB = dbsetup("uno", "AccumuloConfig.jl")

24 Jun 2019 13:58:44,418 WARN - ClientConfiguration.loadFromSearchPath(227) -  Found no client.conf in default paths. Using default client configuration values.


D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x000000000526dd28))

In [15]:
ls(DB)

7-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "mytable_A"           
 "mytable_B"           
 "mytable_C"           
 "trace"               

Once you have the `DBserver` struct, you can create a new table or connect to an existing table:

In [16]:
Tedge = DB["mytable_Tedge","mytable_TedgeT"]
TedgeDeg = DB["mytable_TedgeDeg"]
dtable = DB["table_to_delete"]

ls(DB)

Creating mytable_Tedge in uno
Creating mytable_TedgeT in uno
Creating mytable_TedgeDeg in uno
Creating table_to_delete in uno


11-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "mytable_A"           
 "mytable_B"           
 "mytable_C"           
 "mytable_Tedge"       
 "mytable_TedgeDeg"    
 "mytable_TedgeT"      
 "table_to_delete"     
 "trace"               

You can delete tables just as easily.

In [8]:
delete(dtable)
ls(DB)

7-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "mytable_Tedge"       
 "mytable_TedgeDeg"    
 "mytable_TedgeT"      
 "trace"               

# Inserting Data

Let's insert some data, a csv of tracking logs of the OpenEdx course platform. 

In [9]:
fname = "AccumuloSummit2018/data/entity.csv"
A = ReadCSV(fname)

(1,doc)	19960902_27945.txt
(1,entity)	africa
(1,position)	730;1176;
(1,type)	LOCATION


In [11]:
printFull(A)

47090×5 Array{Union{AbstractString, Number},2}:
      ""  "doc"                  "entity"        "position"     "type"    
     1    "19960825_13108.txt"   "addis ababa"   "54;132;974;"  "LOCATION"
     2    "19960930_84704.txt"   "addis ababa"   "60;"          "LOCATION"
     3    "19961004_96087.txt"   "addis ababa"   "61;305;"      "LOCATION"
     4    "19961006_98377.txt"   "addis ababa"   "68;"          "LOCATION"
     5    "19961009_104796.txt"  "addis ababa"   "59;443;"      "LOCATION"
     6    "19961010_107656.txt"  "addis ababa"   "61;"          "LOCATION"
     7    "19961031_158809.txt"  "addis ababa"   "2109;"        "LOCATION"
     8    "19961101_159647.txt"  "addis ababa"   "1485;"        "LOCATION"
     9    "19961113_185784.txt"  "addis ababa"   "62;"          "LOCATION"
    10    "19960821_6808.txt"    "aden"          "212;"         "LOCATION"
    11    "19961026_145749.txt"  "aden"          "58;"          "LOCATION"
    12    "19961106_169278.txt"  "adriatic sea"  "13

To insert the Associative Array into Accumulo, we just call `put`. Here, we put the associative array into 'Tedge', a table from This will also populate the transpose table of the table pair `Tedge`.

In [12]:
put(Tedge,A)
put(TedgeDeg,sum(A,1)') # what's this?
nnz(Tedge)

188363

# Querying Data

Now that we have data, we can run queries. We can run both row and column queries. By default, a column query on a table pair will query the transpose table, so a column query on a table pair is just as fast as a row query.

In [37]:
@time rowQ = Tedge["20170916-1505593021_0090,",:]
@time colQ = Tedge[:,"event_type|problem_graded,"]

println(size(rowQ))
println(size(colQ))

  0.025368 seconds (1.88 k allocations: 104.813 KiB)
  0.025282 seconds (3.45 k allocations: 211.141 KiB)
(1, 17)
(125, 1)


You can also query multiple keys at once, do range queries, and prefix queries:

In [38]:
twokeys = Tedge["20170916-1505593021_0090,20170911-1505128621_0001,",:]
rangeQ = Tedge["20170911-1505128621_0001,:,20170911-1505128621_0005,",:]
prefixQ = Tedge[:,StartsWith("module_name|")]

println("Query for rows 20170916-1505593021_0090 and 20170911-1505128621_0001:")
println(Row(twokeys))
println("\nQuery for rows 20170911-1505128621_0001 through 20170911-1505128621_0005:")
println(Row(rangeQ))
println("\nQuery for columns that start with module_name|:")
println(Col(prefixQ))

Query for rows 20170916-1505593021_0090 and 20170911-1505128621_0001:
Union{AbstractString, Number}["20170911-1505128621_0001", "20170916-1505593021_0090"]

Query for rows 20170911-1505128621_0001 through 20170911-1505128621_0005:
Union{AbstractString, Number}["20170911-1505128621_0001", "20170911-1505128621_0002", "20170911-1505128621_0003", "20170911-1505128621_0004", "20170911-1505128621_0005"]

Query for columns that start with module_name|:
Union{AbstractString, Number}["module_name|Build Instructions", "module_name|Doppler Processing", "module_name|Example Week 1: Getting Started", "module_name|Final", "module_name|Introduction", "module_name|Introduction to High Performance Scientific Computing", "module_name|Introduction to the LLSC Systems", "module_name|Overview", "module_name|Resources", "module_name|Submitting, Monitoring, and Stopping Jobs", "module_name|TX-E1 Basics", "module_name|The Analyst Challenge", "module_name|The Data Challenge", "module_name|The System Challenge"

# Cleanup

Again, to delete tables, just call `delete`:

In [29]:
delete(Tedge)
delete(TedgeDeg)
ls(DB)

5-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "table_to_delete"     
 "trace"               